In [1]:
!pip install openpyxl

You should consider upgrading via the '/Users/rema/.virtualenvs/ds/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd

# Структура данных
### ID - уникальный идентификатор
### Longitude - долгота
### Latitude - широта
### Name - названия
### AdmArea - административный округ
### District - район
### Address - улица и дом

In [3]:
df = pd.DataFrame(columns = ['Name', 'AdmArea', 'District', 'Address', 'geoData'])

# Религия

In [5]:
religion_rpc = pd.read_json('../data/Religion/Религиозные объекты РПЦ.json', encoding="cp1251")
religion_catholics = pd.read_json('../data/Religion/Католические храмы.json', encoding="cp1251")
religion_mosques = pd.read_json('../data/Religion/Мечети.json', encoding="cp1251")
religion_monasteries = pd.read_json('../data/Religion/Монастыри.json', encoding="cp1251")
religion_synagogues = pd.read_json('../data/Religion/Синагоги.json', encoding="cp1251")

In [6]:
print(religion_rpc.columns, religion_rpc.shape)
print(religion_catholics.columns, religion_catholics.shape)
print(religion_mosques.columns, religion_mosques.shape)
print(religion_monasteries.columns, religion_monasteries.shape)
print(religion_synagogues.columns, religion_synagogues.shape)

Index(['Latitude_WGS84', 'global_id', 'ID', 'Name', 'AdmArea', 'District',
       'Address', 'MetroStation', 'MetroLine', 'PublicPhone', 'WebSite',
       'Longitude_WGS84', 'geoData', 'geodata_center'],
      dtype='object') (551, 14)
Index(['ID', 'ObjectName', 'global_id', 'AdmArea', 'District', 'Address',
       'PublicPhone', 'Fax', 'Email', 'WebSite', 'geoData', 'geodata_center'],
      dtype='object') (3, 12)
Index(['ID', 'ObjectName', 'AdmArea', 'global_id', 'District', 'Address',
       'PublicPhone', 'Fax', 'Email', 'WebSite', 'geoData', 'geodata_center'],
      dtype='object') (4, 12)
Index(['ID', 'NameOfReligiousOrganization', 'global_id', 'AdmArea', 'District',
       'Address', 'PublicPhone', 'WebSite', 'geoData', 'geodata_center'],
      dtype='object') (15, 10)
Index(['ID', 'ObjectName', 'AdmArea', 'global_id', 'District', 'Address',
       'PublicPhone', 'Fax', 'Email', 'WebSite', 'geoData', 'geodata_center'],
      dtype='object') (5, 12)


In [7]:
columns = ['Name', 'AdmArea', 'District', 'Address', 'geoData']
df = df.append(religion_rpc[columns])

In [8]:
religion_catholics.rename(columns={"ObjectName": "Name"}, inplace=True)
df = df.append(religion_catholics[columns])

In [9]:
religion_mosques.rename(columns={"ObjectName": "Name"}, inplace=True)
df = df.append(religion_mosques[columns])

In [10]:
religion_monasteries.rename(columns={"NameOfReligiousOrganization": "Name"}, inplace=True)
df = df.append(religion_monasteries[columns])

In [11]:
religion_synagogues.rename(columns={"ObjectName": "Name"}, inplace=True)
df = df.append(religion_synagogues[columns])

In [12]:
df.shape

(578, 5)

# Транспорт

In [13]:
transport_subway_exits = pd.read_excel(
    '../data/Transport/Входы и выходы вестибюлей станций Московского метрополитена.xlsx')

# Нет адресных данных
# transport_bus_stops = pd.read_json(
#     'data/hackathon/Transport/Маршруты и остановки наземного городского пассажирского транспорта.json', encoding="cp1251")

/Users/rema/.virtualenvs/ds/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [14]:
print(transport_subway_exits.columns, transport_subway_exits.shape)

Index(['ID', 'Name', 'OnTerritoryOfMoscow', 'AdmArea', 'District',
       'Longitude_WGS84', 'Latitude_WGS84', 'VestibuleType', 'NameOfStation',
       'Line', 'CulturalHeritageSiteStatus', 'ModeOnEvenDays', 'ModeOnOddDays',
       'FullFeaturedBPAAmount', 'LittleFunctionalBPAAmount', 'BPAAmount',
       'RepairOfEscalators', 'ObjectStatus', 'global_id', 'geodata_center',
       'geoarea'],
      dtype='object') (1020, 21)


In [15]:
transport_subway_exits["Address"] = pd.NA

In [16]:
transport_subway_exits.rename(columns={"geodata_center": "geoData"}, inplace=True)
df = df.append(transport_subway_exits[columns])

# Культура

In [17]:
сulture_objects = pd.read_json('../data/Culture/Объекты культурного наследия.json', encoding="cp1251")
сulture_museums = pd.read_json('../data/Culture/Музеи.json', encoding="cp1251")

In [18]:
сulture_objects.rename(columns={"ObjectNameOnDoc": "Name", 'Addresses': 'Address'}, inplace=True)
df = df.append(сulture_objects[columns])

In [19]:
сulture_objects.rename(columns={"FullName": "Name", 'Addresses': 'Address'}, inplace=True)
df = df.append(сulture_objects[columns])
сulture_museums

,Category,CommonName,FullName,ShortName,OrgInfo,ObjectAddress,ChiefOrg,ChiefName,ChiefPosition,PublicPhone,...,RarePaperAmount,OtherObjAmount,TotalPreciousObjAmount,WeaponsObjAmount,PaintingsAmount,SculptureAmount,GraphicsAmount,TechHistoryObjAmount,EthnoObjAmount,NatureObjAmount
0,Музей,Мемориальная квартира А. Белого,Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы «ГМП», Мемориальная квартира А....",[{'FullName': 'Государственное бюджетное учреж...,[{'AdmArea': 'Центральный административный окр...,Государственное бюджетное учреждение культуры ...,Богатырев Евгений Анатольевич,директор,[{'PublicPhone': '(499) 241-77-02'}],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Музей,Музей археологии Москвы,Государственное бюджетное учреждение культуры ...,ГБУК города Москвы «Музейное объединение «Музе...,[{'FullName': 'Государственное бюджетное учреж...,[{'AdmArea': 'Центральный административный окр...,Государственное бюджетное учреждение культуры ...,Трапкова Анна Владимировна,генеральный директор,[{'PublicPhone': '(495) 692-00-20'}],...,1.0,187.0,187.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Музей,Музейно - выставочный Центр В.А. Гиляровского,Государственное бюджетное учреждение культуры ...,ГБУК города Москвы «Музейное объединение «Музе...,[{'FullName': 'Государственное бюджетное учреж...,[{'AdmArea': 'Центральный административный окр...,Государственное бюджетное учреждение культуры ...,Трапкова Анна Владимировна,генеральный директор,[{'PublicPhone': '(495) 637-31-35'}],...,1.0,4.0,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Музей,Мемориальный музей космонавтики,Государственное бюджетное учреждение культуры ...,ГБУК г. Москвы «ММК»,[{'FullName': 'Государственное бюджетное учреж...,[{'AdmArea': 'Северо-Восточный административны...,Департамент культуры города Москвы,Артюхина Наталья Витальевна,директор,[{'PublicPhone': '(499) 750-23-00'}],...,NaN,91252.0,NaN,NaN,243.0,80.0,NaN,NaN,NaN,NaN
4,Музей,Выставочный зал Музея русского лубка и наивног...,Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы «МРЛИНИ», Выставочный зал «Наро...",[{'FullName': 'Государственное бюджетное учреж...,[{'AdmArea': 'Восточный административный округ...,Департамент культуры города Москвы,Сергеев Владимир Владимирович,директор,[{'PublicPhone': '(499) 367-45-81'}],...,NaN,151.0,NaN,NaN,NaN,101.0,NaN,NaN,NaN,NaN
5,Музей,Государственный музей А. С. Пушкина,Государственное бюджетное учреждение культуры ...,ГБУК «ГМП»,[{'FullName': 'Государственное бюджетное учреж...,[{'AdmArea': 'Центральный административный окр...,Департамент культуры города Москвы,Богатырев Евгений Анатольевич,директор,[{'PublicPhone': '(495) 637-26-36'}],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Музей,Музейно-выставочный центр «Дача»,Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы «МРЛИНИ», Музейно-выставочный ц...",[{'FullName': 'Государственное бюджетное учреж...,[{'AdmArea': 'Восточный административный округ...,Департамент культуры города Москвы,Сергеев Владимир Владимирович,директор,[{'PublicPhone': '(495) 301-03-48'}],...,NaN,NaN,NaN,NaN,1166.0,101.0,2755.0,NaN,NaN,NaN
7,Музей,"Музейно-выставочное объединение «Манеж», Центр...",Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы «МВО «Манеж», ЦВЗ «Манеж»",[{'FullName': 'Государственное бюджетное учреж...,[{'AdmArea': 'Центральный административный окр...,Департамент культуры города Москвы,Толпина Ирина Александровна,генеральный директор,"[{'PublicPhone': '(499) 394-26-37'}, {'PublicP...",...,NaN,NaN,NaN,NaN,255.0,NaN,NaN,NaN,NaN,NaN
8,Музей,"Мемориальный музей космонавтики, Мемориальный ...",Государственное бюджетное учреждение культуры ...,ГБУК г. Москвы «Мемориальный музей космонавтик...,[{'FullName': 'Государственное бюджетное учреж...,[{'AdmArea': 'Северо-Восточный административны...,Государственное бюджетное учреждение культуры ...,Артюхина Наталья Витальевна,директор,"[{'PublicPhone': '(495) 683-81-97'}, {'PublicP...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Музей,Музей В.А. Тропи

In [121]:
сulture_museums['ObjectAddress'][0]

[{'AdmArea': 'Центральный административный округ',
  'District': 'район Арбат',
  'PostalCode': '119002',
  'Address': 'город Москва, улица Арбат, дом 55/32',
  'Availability': [{'available_k': 'недоступно',
    'available_o': 'полностью',
    'available_z': 'полностью',
    'available_s': 'полностью',
    'available_element': [{'Group_mgn': 'инвалиды-колясочники',
      'Area_mgn': 'пути движения и эвакуации',
      'Element_mgn': 'ширина полосы движения',
      'available_degree': 'недоступно',
      'available_index': 'не соответствует'},
     {'Group_mgn': 'инвалиды-опорники',
      'Area_mgn': 'санитарный узел',
      'Element_mgn': 'поручни у унитаза',
      'available_degree': 'частично',
      'available_index': 'нет'},
     {'Group_mgn': 'инвалиды-опорники',
      'Area_mgn': 'входная группа',
      'Element_mgn': 'поручни на имеющейся лестнице',
      'available_degree': 'полностью',
      'available_index': 'нет'},
     {'Group_mgn': 'инвалиды по зрению',
      'Area_mgn': '

In [122]:
df.head()

,Name,AdmArea,District,Address,geoData
0,Храм святого равноапостольного князя Владимира...,Юго-Восточный административный округ,район Кузьминки,"улица Маршала Чуйкова, дом 30","{'coordinates': [37.758455, 55.695289], 'type'..."
1,Храм Влахернской иконы Божией Матери в Кузьминках,Юго-Восточный административный округ,район Выхино-Жулебино,"улица Кузьминская дом 7, строение 1","{'coordinates': [37.790386, 55.690614], 'type'..."
2,Часовня иконы Божией Матери «Всех скорбящих Ра...,Юго-Восточный административный округ,район Некрасовка,"улица 2-я Вольская, дом 19, корпус 1","{'coordinates': [37.933154, 55.694518], 'type'..."
3,Храм святого великомученика Георгия Победоносц...,Юго-Восточный административный округ,район Выхино-Жулебино,"Волгоградский проспект, дом 168","{'coordinates': [37.812268, 55.68983], 'type':..."
4,"Храм Елисаветы прмц. при школе-интернате № 55,...",Юго-Восточный административный округ,район Кузьминки,"улица Маршала Чуйкова, дом 26, строение 1","{'coordinates': [37.758028, 55.698198], 'type'..."
